In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [3]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller

In [4]:
np.random.seed(0)

translator = CirqTranslater(3, untouchable_blocks = [1])

translator.env_qubits = [2]
translator.encoder_id = 0
translator.channel_id = 1
translator.decoder_id = 2


simplifier = Simplifier(translator)


etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)

killer = GateKiller(translator, mode="discrimination", params = etas)


inserter = IdInserter(translator.n_qubits, untouchable_blocks=translator.channel_id)


args_evaluator = {"n_qubits":translator.n_qubits, "problem":"acd","params":etas}
evaluator = Evaluator(args=args_evaluator, lower_bound_cost=minimizer.lower_bound_cost, nrun=0)


In [5]:
u1_layer_encoder = u1_layer(translator, inds= [0,1], block_id=translator.encoder_id)
channel_db = amplitude_damping_db(translator, qubits_ind=[1,2], eta=1, block_id = translator.channel_id)
u1_layer_decoder = u1_layer(translator, inds = [0,1], block_id=translator.decoder_id)

u2_layer_encoder = u2_db(translator, 0,1, block_id=translator.encoder_id)
channel_db = amplitude_damping_db(translator, qubits_ind=[1,2], eta=1, block_id = translator.channel_id)
u2_layer_decoder = u2_db(translator, 0,1, block_id=translator.decoder_id)


circuit_db = concatenate_dbs([u1_layer_encoder, channel_db, u1_layer_decoder])
#circuit_db = concatenate_dbs([u2_layer_encoder, channel_db, u2_layer_decoder])

circuit, circuit_db = translator.give_circuit(circuit_db)
circuit

(0, 0): ───Rz(th_0)───Rx(th_1)───Rz(th_2)───Rz(th_11)───Rx(th_12)───Rz(th_13)─────────────────────────────────────────────────────────────────

(0, 1): ───Rz(th_3)───Rx(th_4)───Rz(th_5)───H───────────X────────────────────────X───H────────────────X───Rz(th_14)───Rx(th_15)───Rz(th_16)───
                                                        │                        │                    │
(0, 2): ───Rz(th_6)───Rx(th_7)───Rz(th_8)───H───────────@───────────Ry(0.318π)───@───H───Ry(0.318π)───@───Rz(th_17)───Rx(th_18)───Rz(th_19)───

In [6]:
batch_circuits, trainable_symbols, trainable_params_value = prepare_optimization_discrimination(translator, circuit_db, etas)
cost, resolver, training_history = minimizer.minimize(batch_circuits, symbols = trainable_symbols, parameter_values = trainable_params_value )
circuit_db = translator.update_circuit_db_param_values(circuit_db, resolver)
original_db = circuit_db.copy()


circuit_db = original_db.copy()

evaluator.add_step(circuit_db, cost, relevant=True)
evaluator.save_dicts_and_displaying()

circuit_db, cost = killer.remove_irrelevant_gates(cost, circuit_db)
evaluator.add_step(circuit_db, cost, relevant=False)
evaluator.save_dicts_and_displaying()


Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
kill 1qbit gate, try 0/18. Increased by: 1.4540061101797619e-06%
kill 1qbit gate, try 1/18. Increased by: 1.4540061101797619e-06%
kill 1qbit gate, try 2/18. Increased by: 1.4540061101797619e-06%
kill 1qbit gate, try 3/18. Increased by: 1.4540061101797619e-06%
kill 1qbit gate, try 4/18. Increased by: 2.1810092221130617e-06%
kill 1qbit gate, try 5/18. Increased by: 7.270030550898809e-07%
kill 1qbit gate, try 6/18. Increased by: 0.0%
kill 1qbit gate, try 7/18. Increased by: -7.270030550898809e-07%
kill 1qbit gate, try 8/18. Increased by: 0.0%
kill 1qbit gate, try 9/18. Increased by: 0.0%


In [8]:
simplifier = Simplifier(translator)

In [10]:
for vans_it in range(3):
    print("vans iteration: {}".format(vans_it))
    mutated_circuit_db = inserter.insert_many_mutations(circuit_db )
    simplified_db, nreds = simplifier.reduce_circuit(mutated_circuit_db)

    batch_circuits, trainable_symbols, trainable_params_value = prepare_optimization_discrimination(translator, simplified_db, etas)
    
    cost, resolver, training_history = minimizer.minimize(batch_circuits, symbols = trainable_symbols, parameter_values = trainable_params_value )
    mutation_db = translator.update_circuit_db_param_values(mutated_circuit_db, resolver)    

    if evaluator.accept_cost(cost):
        circuit_db = mutation_db
        circuit_db, cost = killer.remove_irrelevant_gates(cost, circuit_db)
        evaluator.add_step(circuit_db, cost, relevant=False)
        evaluator.save_dicts_and_displaying()

vans iteration: 0
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f40e0296978>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f40e0296978>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f40e0296978>>
kill 1qbit gate, try 0/15. Increased by: 0.0%
vans iteration: 1
simplified using  <bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7f40e0296978>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f40e0296978>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f40e0296978>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f40e0296978>>
simplified using  <bound method Simplifier.rule_3 of <utilities.simplifier.Simplifier object at 0x7f40e0296978>>
kill 1qbit gat